In [1]:
!pip install ultralytics -q

In [17]:
!nvidia-smi

Thu Dec 18 20:46:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   76C    P0             42W /   70W |     667MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import pandas as pd
import numpy as np
import os
import shutil
import cv2
from tqdm.auto import tqdm
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import yaml

# Configuration
INPUT_DIR = '/kaggle/input/solidworks-ai-hackathon'
WORK_DIR = '/kaggle/working'
DATASET_DIR = os.path.join(WORK_DIR, 'yolo_dataset')
IMG_SIZE = 640  # Standard YOLO image size
BATCH_SIZE = 16
EPOCHS = 20      # Increase to 50+ for better results

# Class mapping (Order matters for the submission file!)
# The submission requires: bolt, locatingpin, nut, washer
CLASSES = ['bolt', 'locatingpin', 'nut', 'washer']
CLASS_MAP = {name: idx for idx, name in enumerate(CLASSES)}

print("Classes mapped as:", CLASS_MAP)

Classes mapped as: {'bolt': 0, 'locatingpin': 1, 'nut': 2, 'washer': 3}


# Data Preporcessing CSV to YOLO

In [3]:
# 1. Prepare Directories
for split in ['train', 'val']:
    os.makedirs(os.path.join(DATASET_DIR, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(DATASET_DIR, 'labels', split), exist_ok=True)

# 2. Load Metadata
bboxes_df = pd.read_csv(os.path.join(INPUT_DIR, 'train_bboxes.csv'))
labels_df = pd.read_csv(os.path.join(INPUT_DIR, 'train_labels.csv'))

# Get list of all unique training images
all_images = labels_df['image_name'].unique()

# Split into Train and Validation
train_imgs, val_imgs = train_test_split(all_images, test_size=0.2, random_state=42)

def process_dataset(image_list, split_name):
    print(f"Processing {split_name} data...")
    
    for img_name in tqdm(image_list):
        # Source path
        src_img_path = os.path.join(INPUT_DIR, 'train/train', img_name)
        
        # Check if image exists
        if not os.path.exists(src_img_path):
            continue
            
        # Read Image to get dimensions (Synthetic images are likely uniform, but safer to check)
        img = cv2.imread(src_img_path)
        h, w, _ = img.shape
        
        # Copy image to working directory
        dst_img_path = os.path.join(DATASET_DIR, 'images', split_name, img_name)
        shutil.copy(src_img_path, dst_img_path)
        
        # Get bboxes for this image
        img_bboxes = bboxes_df[bboxes_df['image_name'] == img_name]
        
        # Prepare Label File content
        label_path = os.path.join(DATASET_DIR, 'labels', split_name, img_name.replace('.png', '.txt'))
        
        with open(label_path, 'w') as f:
            # If no bboxes, file remains empty (which is correct for background-only images)
            for _, row in img_bboxes.iterrows():
                cls_name = row['class']
                if cls_name not in CLASS_MAP: continue
                
                cls_id = CLASS_MAP[cls_name]
                
                # Convert xyxy to xywh normalized
                # x_min, y_min, x_max, y_max provided in CSV
                x_center = ((row['x_min'] + row['x_max']) / 2) / w
                y_center = ((row['y_min'] + row['y_max']) / 2) / h
                width = (row['x_max'] - row['x_min']) / w
                height = (row['y_max'] - row['y_min']) / h
                
                f.write(f"{cls_id} {x_center} {y_center} {width} {height}\n")

# Process both splits
process_dataset(train_imgs, 'train')
process_dataset(val_imgs, 'val')

Processing train data...


  0%|          | 0/8000 [00:00<?, ?it/s]

Processing val data...


  0%|          | 0/2000 [00:00<?, ?it/s]

In [4]:
yaml_content = f"""
path: {DATASET_DIR}
train: images/train
val: images/val
names:
  0: bolt
  1: locatingpin
  2: nut
  3: washer
"""

with open('data.yaml', 'w') as f:
    f.write(yaml_content)

In [13]:
# Load a pretrained model
model = YOLO('yolov8s.pt') 

# Train
results = model.train(
    data='data.yaml',
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    name='solidworks_parts',
    device=[0,1],
    verbose=True
)

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=solidworks_parts3, nbs=64, nms=False, opset=None, optimize=False, optimizer

In [15]:
# Directory containing test images
TEST_IMG_DIR = os.path.join(INPUT_DIR, 'test/test')
test_images = [f for f in os.listdir(TEST_IMG_DIR) if f.endswith('.png')]

submission_rows = []
print("Running inference on test set...")

# Run inference in batches (or one by one)
# stream=True helps with memory on large datasets
results = model.predict(source=TEST_IMG_DIR, stream=True, conf=0.25, verbose=False)

for result in results:
    img_name = os.path.basename(result.path)
    
    # Initialize counts
    counts = {
        'bolt': 0,
        'locatingpin': 0,
        'nut': 0,
        'washer': 0
    }
    
    # result.boxes.cls contains the class IDs of detections
    # Move to CPU and convert to numpy for counting
    det_classes = result.boxes.cls.cpu().numpy().astype(int)
    
    for cls_id in det_classes:
        # Map ID back to name using the CLASSES list
        # 0->bolt, 1->locatingpin, 2->nut, 3->washer
        if cls_id < len(CLASSES):
            class_name = CLASSES[cls_id]
            counts[class_name] += 1
            
    # Prepare row
    row = {
        'image_name': img_name,
        'bolt': counts['bolt'],
        'locatingpin': counts['locatingpin'],
        'nut': counts['nut'],
        'washer': counts['washer']
    }
    submission_rows.append(row)

# Create DataFrame
submission_df = pd.DataFrame(submission_rows)

# Reorder columns to match requirement exactly
submission_df = submission_df[['image_name', 'bolt', 'locatingpin', 'nut', 'washer']]

# Save submission
submission_df.to_csv('/kaggle/working/submission2.csv', index=False)
print("Submission.csv created successfully!")
print(submission_df.head())

Running inference on test set...
Submission.csv created successfully!
                             image_name  bolt  locatingpin  nut  washer
0  0040313aa7c7478c8ca264bf573c53fe.png     1            3    0       0
1  0080e45f4375443b96ee81dc04075117.png     2            0    0       1
2  0086da1a1914469caa4d042f8e4e94ef.png     0            0    0       1
3  009ccd5b64a043e0afbae0f840f7cfd4.png     0            0    0       1
4  00a522a027b24fcfb8ee0857655b953d.png     0            1    0       0
